## Amazon Lookout for Vision Lab

모델 만드는 법 익히는 걸 도와주기 위해 Amazon Lookout for Vision은 사용 가능한 회로 기판 (circuit_board) 이미지 예제를 제공합니다. 해당 이미지는 https://docs.aws.amazon.com/lookout-for-vision/latest/developer-guide/su-prepare-example-images.html 에서 가져온 것입니다.

### 환경 변수

첫 번째 단계에서 이 노트북에 필요한 두 개의 전역 변수를 정의하려고 합니다.

- 버킷: Amazon Lookout for Vision 소스로 생성하여 사용할 S3 버킷
    - 참고: 주석을 주의 깊게 읽어주세요. 지역에 맞는 올바른 명령이 될 수 있게 주석 처리 제거를 해야할 경우가 있습니다.
- 프로젝트: Amazon Lookout for Vision에서 사용할 프로젝트 명

In [1]:
import os
import boto3

BUCKET = "lfv-s3-bucket-<youralias>-MMDDYY"
PROJECT = "circuitproject"

os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = boto3.session.Session().region_name

client = boto3.client("lookoutvision")

여기에서 지역을 확인할 수 있습니다.

In [2]:
# 지역을 확인하세요.
print(boto3.session.Session().region_name)

us-east-1


지역에 따라 다음 셀의 지침을 따르세요.

In [5]:
## 기존 S3 버킷을 사용하지 않는 경우 S3 버킷을 새로 생성합니다.

if boto3.session.Session().region_name == "us-east-1":
    _ = !aws s3api create-bucket --bucket $BUCKET
else:
    _ = !aws s3api create-bucket --bucket $BUCKET --create-bucket-configuration LocationConstraint=$REGION

## 이미지 준비와 EDA

Amazon Lookout for Vision을 위해 - 다음도 살펴보세요.

* https://aws.amazon.com/lookout-for-vision/ 과
* https://aws.amazon.com/blogs/aws/amazon-lookout-for-vision-new-machine-learning-service-that-simplifies-defect-detection-for-manufacturing/ 이 예제처럼 사전에 라벨링된 이미지를 사용하는 경우 학습과 검증 용도를 구분하여 폴더 구조를 생성할 수 있습니다. 또한 해당 폴더 (정상 = 좋음, 이상 = 나쁨)를 통해 Amazon Lookout에 대한 이미지 라벨을 지정합니다. 

AWS Lookout of Vision에서 제공하는 샘플 이미지를 가져옵니다. 만약 사용자 본인의 이미지를 사용하려는 경우 이 단계에서 따로 준비합니다. 

### *manifest* 파일 생성하기

Amazon SageMaker Ground Truth를 사용한 적이 있다면 Manifest 파일이 익숙할 것입니다. 만약 사용해본 적이 없더라도 크게 신경쓸 필요 없습니다. 

그러나 좀 더 구체적인 내용이 궁금하다면 아래 내용을 계속 읽어보세요.

각 훈련 / 검증 데이터셋은 Manifest 파일이 필요합니다. 이 파일은 Amazon Lookout for Vision에서 이미지 탐색 위치를 결정하는데 사용됩니다. Manifest는 고정적인 구조를 따릅니다. 가장 중요한 것은 키 (JSON 형식)입니다. 즉 *source-ref*는 각 파일의 위치, *auto-label*은 각 레이블 값 (0 = 나쁨, 1 = 좋음), *folder*는 Amazon Lookout이 훈련 또는 검증을 구분하여 사용하도록, *creation-date*는 이미지가 저장된 시간을 알 수 있도록 합니다. 다른 모든 필드값은 미리 설정되어 있습니다.

각 Manifest 파일 자체에는 N개의 JSON 개체가 포함되어 있습니다. 여기서 N은 해당 데이터셋에서 사용되는 이미지 개수입니다.

In [13]:
# 일시 생성을 위한 datetime과 대응하는 파일에 JSON 객체를 덤프하기 위한 json 모듈 임포트
from datetime import datetime
import json

# Manifest 파일 포맷 안의 현재 일자와 시간
now = datetime.now()
dttm = now.strftime("%Y-%m-%dT%H:%M:%S.%f")

# 두 종류의 데이터셋: 훈련과 테스트
datasets = ["train", "test"]

# 각 데이터셋마다...
for ds in datasets:
    # ...이용 가능한 폴더들 (정상 또는 이상)을 List로 만듭니다. 
    # print(ds)
    folders = os.listdir("./circuitboard/{}".format(ds))
    # 그런 다음 해당 데이터셋을 위해 Manifest 파일을 엽니다...
    with open("{}.manifest".format(ds), "w") as f:
        for folder in folders:
            filecount = 0
            # print(folder)
            # ...그리고 처음 만든 List를 이용해 두 폴더를 순회합니다.
            # 대응 파일과 적절한 라벨을 설정
            # (위에서 언급했듯이: 1 = 좋음, 0 = 나쁨):
            files = os.listdir("./circuitboard/{}/{}".format(ds, folder))
            label = 1
            if folder == "anomaly":
                label = 0
            # 폴더 안의 각 파일마다...
            for file in files:
                filecount += 1
                # print(filecount)
                # 전체 데이터셋을 사용하려면 다음 두 행을 주석 처리하세요. 
                # if filecount > 20:
                #     break
                # ...Manifest JSON 객체를 생성하고 Manifest 파일에 저장합니다.
                # 새 줄을 생성하기 위해 '/n'을 추가하는 것 잊지 마세요.
                manifest = {
                  "source-ref": "s3://{}/{}/{}/{}/{}".format(BUCKET, PROJECT, ds, folder, file)
                  , "auto-label": label
                  , "auto-label-metadata": {
                    "confidence": 1
                    , "job-name": "labeling-job/auto-label"
                    , "class-name": folder
                    , "human-annotated": "yes"
                    , "creation-date": dttm
                    , "type": "groundtruth/image-classification"
                  }
                }
                f.write(json.dumps(manifest) + "\n")

### S3에 Manifest 파일과 이미지 업로드하기

이제 모든 이미지와 Manifest 파일을 업로드할 차례입니다. 

In [7]:
# S3 버킷에 Manifest 파일을 업로드합니다.
!aws s3 cp train.manifest s3://{BUCKET}/{PROJECT}/train.manifest --quiet 
!aws s3 cp test.manifest s3://{BUCKET}/{PROJECT}/test.manifest --quiet 

In [8]:
# S3 버킷에 이미지 파일을 업로드합니다.
!aws s3 cp circuitboard/train/normal s3://{BUCKET}/{PROJECT}/train/normal --quiet --recursive
!aws s3 cp circuitboard/train/anomaly s3://{BUCKET}/{PROJECT}/train/anomaly --quiet --recursive

!aws s3 cp circuitboard/test/normal s3://{BUCKET}/{PROJECT}/test/normal --quiet --recursive
!aws s3 cp circuitboard/test/anomaly s3://{BUCKET}/{PROJECT}/test/anomaly --quiet --recursive

## Amazon Lookout for Vision

거의 완료되었습니다. Amazon Lookout 프로젝트 (콘솔, CLI 또는 boto3)를 생성하는 방법으로 몇 가지 옵션이 있습니다. 이 예에서는 boto3 SDK를 선택했습니다. 콘솔을 확인하는 것도 좋습니다. 프로젝트를 생성하고 모델을 훈련시키는 방법은 매우 간단합니다. 이건 우리가 고객에게도 꼭 보여줘야 할 내용입니다!

SDK로 수행하는 단계는 다음과 같습니다.

1. 프로젝트 생성하기 (처음에 설정한 이름)
2. 훈련 데이터셋 탐색 위치를 프로젝트에 알려줍니다. 이는 훈련 용 Manifest 파일을 통해 수행됩니다.
3. 테스트 데이터셋 탐색 위치를 프로젝트에 알려줍니다. 이는 테스트 용 Manifest 파일을 통해 수행됩니다.
    - 참고: 이 단계는 선택 사항입니다. 일반적으로 모든 '테스트' 관련 코드는 선택 사항입니다. Amazon Lookout for Vision을 '훈련' 데이터셋만 이용하여 동작시킬 수 있습니다. AI / ML 모델을 훈련할 때 훈련과 테스트 모두 이용하는 편이 일반적인 (모범) 관행이므로 둘 다 사용하기로 결정했습니다. 그리고 고객이 다음 단계로 나아갈 때 도움이 되게끔 항상 고객에게 이를 알려주세요.
4. 모델을 만듭니다. 이 명령은 모델 훈련과 검증 작업을 촉발합니다.

**참고**: 모델 훈련 작업은 뒷단에서 딥러닝을 사용하므로 몇 시간이 걸릴 수 있습니다. 모델이 훈련되면 해당 노트북에서 계속 사용하면서 예측할 수 있습니다.

### 프로젝트 생성하기

In [11]:
# 프로젝트 생성하기
print("Creating project:" + PROJECT)
response = client.create_project(ProjectName=PROJECT)
# print("Project ARN: " + response["ProjectMetadata"]["ProjectArn"])
print("Done!")

Creating project:circuitproject
Done!


### 훈련 데이터셋 생성하기

In [15]:
# 훈련 데이터셋 생성하기
dataset_type = "train"
manifest_file = PROJECT + "/train.manifest"

print("Creating dataset...")
dataset = json.loads('{"GroundTruthManifest": { "S3Object": { "Bucket": "' + BUCKET + '", "Key": "' + manifest_file + '"}}}')

response = client.create_dataset(ProjectName=PROJECT, DatasetType=dataset_type, DatasetSource=dataset)
print("Dataset Status: " + response["DatasetMetadata"]["Status"])
print("Dataset Status Message: " + response["DatasetMetadata"]["StatusMessage"])
print("Dataset Type: " + response["DatasetMetadata"]["DatasetType"])
print("Done!")

Creating dataset...
Dataset Status: CREATE_IN_PROGRESS
Dataset Status Message: The dataset is creating.
Dataset Type: train
Done!


### 테스트 데이터셋 생성하기

In [17]:
# 테스트 데이터셋 생성하기
dataset_type = "test"
manifest_file = PROJECT + "/test.manifest"

print("Creating dataset...")
dataset = json.loads('{"GroundTruthManifest": {"S3Object": {"Bucket": "' + BUCKET + '", "Key": "' + manifest_file + '"}}}')

response = client.create_dataset(ProjectName=PROJECT, DatasetType=dataset_type, DatasetSource=dataset)
print("Dataset Status: " + response["DatasetMetadata"]["Status"])
print("Dataset Status Message: " + response["DatasetMetadata"]["StatusMessage"])
print("Dataset Type: " + response["DatasetMetadata"]["DatasetType"])
print("Done!")

Creating dataset...
Dataset Status: CREATE_IN_PROGRESS
Dataset Status Message: The dataset is creating.
Dataset Type: test
Done!


### 모델 생성 / 훈련하기

In [18]:
# 모델 생성 / 훈련하기
output_bucket = BUCKET
output_folder = PROJECT + "/model/"

print("Creating model...")
output_config = json.loads('{"S3Location": {"Bucket": "' + output_bucket + '", "Prefix": "' + output_folder + '"}}')

response = client.create_model(ProjectName=PROJECT, OutputConfig=output_config)
# print("ARN: " + response["ModelMetadata"]["ModelArn"])
print("Version: " + response["ModelMetadata"]["ModelVersion"])
print("Status: " + response["ModelMetadata"]["Status"])
print("Message: " + response["ModelMetadata"]["StatusMessage"])
print("Done!")

Creating model...
ARN: arn:aws:lookoutvision:us-east-1:998601677581:model/circuitproject/1
Version: 1
Status: TRAINING
Message: The model is being trained.
Done!


### 모델 배포

운영 단계로 모델을 가져오는 작업은 "시작"이라고 알려주는 것만큼이나 쉽습니다. 해당 프로세스도 몇 분 정도 걸립니다. 그러니 기다려주세요. 콘솔 (또는 CLI를 통해)에서 모델 상태를 재확인할 수 있습니다. 

#### 모델 훈련이 완료될 때까지 기다립니다

In [ ]:
import time

while client.describe_model(ProjectName=PROJECT, ModelVersion="1")["ModelDescription"]["Status"] != "TRAINED":
    print(".", end=""); time.sleep(5);
print("Done!")

#### 훈련시킨 모델 호스팅하기

In [ ]:
model_version = "1"
min_inference_units = 1 
    
print("Starting model version " + model_version + " for project " + PROJECT)
response = client.start_model(ProjectName=PROJECT, ModelVersion=model_version, MinInferenceUnits=min_inference_units)
print("Status: " + response["Status"])

#### 모델 호스팅이 완료될 때까지 기다립니다

In [ ]:
while client.describe_model(ProjectName=PROJECT, ModelVersion="1")["ModelDescription"]["Status"] != "HOSTED":
    print(".", end=""); time.sleep(5);
print("Done!")

### 예측해보기

boto3 SDK를 통해 예측 작업을 하려면 프로젝트 명, 모델 버전, 콘텐츠 유형과 샘플 이미지가 필요합니다. 본 SageMaker 노트북 인스턴스는 로컬 이미지를 사용할 것입니다.

GUI 기반 솔루션을 사용하여 예측 작업을 하려면 다음의 데모를 참조하십시오. - https://github.com/aws-samples/amazon-lookout-for-vision-demo

In [ ]:
# 추가 이미지에서 비정상 이미지 선택
photo = "circuitboard/extra_images/extra_images-anomaly_3.jpg"
model_version = "1"
    
with open(photo, "rb") as image:
    response = client.detect_anomalies(ProjectName=PROJECT, ContentType="image/jpeg", Body=image.read(), ModelVersion=model_version)
print("Anomalous?: " + str(response["DetectAnomalyResult"]["IsAnomalous"]))
print("Confidence: " + str(response["DetectAnomalyResult"]["Confidence"]))

In [ ]:
# 추가 이미지에서 정상 이미지 선택
photo = "circuitboard/extra_images/extra_images-normal_1.jpg"
model_version = "1"
    
with open(photo, "rb") as image:
    response = client.detect_anomalies(ProjectName=PROJECT, ContentType="image/jpeg", Body=image.read(), ModelVersion=model_version)
print("Anomalous?: " + str(response["DetectAnomalyResult"]["IsAnomalous"]))
print("Confidence: " + str(response["DetectAnomalyResult"]["Confidence"]))

# 근검 절약하세요, 모델을 중단하세요

모델이 더 이상 필요하지 않으면 중지해서 비용을 절약하세요!

In [ ]:
# 모델이 더 이상 필요하지 않으면 중지해서 비용을 절약하세요!
model_version = "1"

print("Stopping model version " + model_version + " for project " + PROJECT)
response = client.stop_model(ProjectName=PROJECT, ModelVersion=model_version)
print("Status: " + response["Status"])